In [3]:
from bs4 import BeautifulSoup
import glob
import re
import csv
import spacy
from nltk import sent_tokenize
import pandas as pd
nlp = spacy.load('en_core_web_lg')

In [4]:
Tmp = []
def Filter_Title(sen):
    flag = False
    doc = nlp(sen)
    for token in doc:
        if token.pos_=='VERB':
            flag = True
    return flag

In [5]:
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('ner-ontonotes')
def sentence_ner(sen):
    sentence = Sentence(sen, use_tokenizer=True)
    tagger.predict(sentence)
    ner_tmp ,ner_words,ner_tags = [],[],[]
    for token in sentence:
        if token.get_tag('ner').find('ORG')>=0:
            ner_words.append(token.text)
            ner_tags.append(token.get_tag('ner'))
    index = 0
    flag = False
    flag_i = False
    for word , tag in zip(ner_words , ner_tags):
        if  tag =='B-ORG':
            flag = True
            ner_tmp.append(word)
        if flag:
            if 'I-ORG' == tag:
                ner_tmp[index] = ner_tmp[index] +' '+ word
                flag_i = True         
        if flag_i:
            if 'E-ORG' == tag:
                ner_tmp[index] = ner_tmp[index] +' '+ word
                flag , flag_i = False , False
                index += 1     
        if tag == 'S-ORG':
            ner_tmp.append(word)
            index+=1
    return ner_tmp

In [6]:
def nltk_sentence_token(Content):
    Sen= []
    sen_token = sent_tokenize(Content)
    for sen in sen_token:
        if len(sen.split(' '))>=5 and Filter_Title(sen):
            Sen.append(sen)
    return Sen
def Deal_html_return_sen(html_list):
    Sen = []
    for path_html in html_list:
        file = open(path_html , 'r')
        soup = BeautifulSoup(file , 'html.parser')
        file.close()
        tmpp = soup.find_all('p')
        After_filter_title_sen = [i.text.strip() for i in tmpp if i.text and len(i.text.split(' '))>=6 and Filter_Title(i.text)]
        Content = ' '.join(After_filter_title_sen)
        Sen+=nltk_sentence_token(Content)
    return Sen

In [7]:
def Load_WhilteBlack(path):
    b_list = []
    w_list = []
    df_e = pd.read_csv(path)
    for e , label in zip(df_e['Entity'] , df_e['Label']):
        e=e.strip().replace('\ue5b8','')
        label = str(label)
        if label=='1':
            w_list.append(e)
        else:
            b_list.append(e) 
    w_list = list(set(w_list))
    b_list = list(set(b_list))
    return (w_list , b_list)

In [8]:
def Find_NER_In_Sentence_Flair(Total_sen , w_list):
    from flashtext import KeywordProcessor
    keyword_processor = KeywordProcessor()
    for i in w_list:keyword_processor.add_keyword(i)
    Entity = []
    for s in Total_sen:
        Flair = sentence_ner(s)
        w = keyword_processor.extract_keywords(s)
        total = list(set(w+Flair))
        result = []
        for i in total:
            i=i.replace('(','').replace(')','').replace('"','').strip()
            if i in b_list:
                pass
            if i.split(' ')[0].lower()=='the':
                pass
            else:
                result.append(i)
        result = list(set(result))
        Entity+=result
    Entity = list(set(Entity))
    return Entity

In [19]:
def use_black_list(Entity , b_list):
    Entity = [e for e in Entity if e[0].istitle() ]
    T_E = []
    for i in Entity:
        try:
            if i not in b_list:
                if i[-2]==' ':
                    i=i.rstrip('.').strip()+'.'
                    T_E.append(i)
                else:
                    T_E.append(i)
            else:pass
        except:
            pass
    Ent = list(set(T_E))
    return Ent

In [10]:
def Use_whilte_check_flair(Ent,w_list):
    Tmp_e = []
    bin_go=[]
    for e in Ent:
        if e in w_list:
            bin_go.append('O')
        else:
            bin_go.append('X')
        Tmp_e.append(e)
    E = [(b , e)for b , e in zip(Tmp_e , bin_go)]
    return E

In [11]:
html_list = glob.glob('xxxx/*/html/*.html')

In [12]:
Total_sen = Deal_html_return_sen(html_list[:2])

In [13]:
print(len(Total_sen))

379


In [14]:
w_list , b_list = Load_WhilteBlack('Entity.csv')

In [15]:
Entity_tmp = Find_NER_In_Sentence_Flair(Total_sen,w_list)

In [20]:
Final_Entity = use_black_list(Entity_tmp , b_list)

In [21]:
result = Use_whilte_check_flair(Final_Entity , w_list)

In [22]:
import csv
with open('123.csv','w') as f:
    writer = csv.writer(f)
    writer.writerows([('Entity','white_list')])
    writer.writerows(result)